[⬅️ Back to Intro Notebook](https://colab.research.google.com/github/lrhgit/viscoelastic-fluids-demo/blob/main/intro_viscoelasticity.ipynb)


<details>
<summary><b>🧭 Features — click to expand</b></summary>

This version adds classroom-friendly controls:

- **Optional time scaling**: toggle between dimensionless time (*Scale time by* $\tau$) where the simulation spans $T_{\text{process}}\,\tau$, and **fixed physical time** $T_{\max}$ in seconds.
- **Optional y-axis scaling**: stress axis is **globally fixed** (clear amplitude changes); creep axis is **semi‑fixed** to the current window (uses $t_{\max}$).
- **Three-column layout** for parameters.

**Symbols and scales**

| Symbol | Meaning | Control |
|:--|:--|:--|
| $E$ | Elastic modulus | slider |
| $\eta$ | Viscosity | slider |
| $\tau = \eta/E$ | Relaxation time | computed |
| $T_{\text{process}}$ | Duration in units of $\tau$ | slider |
| $T_{\max}$ | Absolute max time (seconds) | slider |
| $De = 1/T_{\text{process}}$ | Deborah number | printed |
| $\gamma_0,\;\sigma_0$ | Imposed strain / stress | sliders |

> 💡 *Tip:* Turn **off** “Scale time by $\tau$” to compare different materials over the same physical time window.

</details>


# Maxwell Model — Viscoelastic Fluid (Series)

Many real materials — such as polymers, biological tissues, and asphalt — exhibit both *elastic* and *viscous* behavior.  
The **Maxwell model** represents a spring and a dashpot in **series** and captures stress relaxation and unbounded creep.

<p align="center">
    <img src="figures/maxwell_model.png" width="450px"><br>
  <em>Figure 1. Maxwell model (spring and dashpot in series)</em>
</p>
$$
\sigma + \tau \dot{\sigma} = \eta \dot{\gamma}, \qquad \tau = \frac{\eta}{E}
$$

Where:  
- $ \sigma $ — shear stress  
- $ \dot{\gamma} $ — shear rate  
- $ E $ — elastic modulus (spring stiffness)  
- $ \eta $ — viscosity (dashpot resistance)  
- $ \tau = \eta / E $ — relaxation time


## 🔹 Stress relaxation and creep – Maxwell Model


In [2]:
#@title 🎛️ Maxwell model-interactive
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import VBox, HBox
import numpy as np

# Slider bounds (also used to compute fixed ranges)
E_min, E_max       = 1e3, 1e7        # Pa
eta_min, eta_max   = 1e4, 1e8        # Pa·s
gamma0_min, gamma0_max = 1e-3, 1e-1  # [-]
sigma0_min, sigma0_max = 1e-1, 1e3   # Pa
Tproc_min, Tproc_max   = 0.1, 20.0   # –
Tmax_min, Tmax_max     = 0.1, 1e3    # s

# Widgets (three columns)
E_slider     = widgets.FloatLogSlider(value=eta_max, base=10, min=np.log10(E_min),   max=np.log10(E_max),   step=0.1, description='E [Pa]')
eta_slider   = widgets.FloatLogSlider(value=1e6, base=10, min=np.log10(eta_min), max=np.log10(eta_max), step=0.1, description='η [Pa·s]')

gamma0_slider = widgets.FloatLogSlider(value=0.5*gamma0_max, base=10, min=np.log10(gamma0_min), max=np.log10(gamma0_max), step=0.1, description='γ₀ [-]')
sigma0_slider = widgets.FloatLogSlider(value=1e1,  base=10, min=np.log10(sigma0_min), max=np.log10(sigma0_max), step=0.1, description='σ₀ [Pa]')

Tproc_slider  = widgets.FloatLogSlider(value=5.0, base=10, min=np.log10(Tproc_min), max=np.log10(Tproc_max), step=0.1, description='T_process [–]')
Tmax_slider   = widgets.FloatLogSlider(value=10.0, base=10, min=np.log10(Tmax_min),  max=np.log10(Tmax_max),  step=0.1, description='T_max [s]')
time_scaled_chk = widgets.Checkbox(value=False, description='Scale time by τ')
fixed_yaxis_chk = widgets.Checkbox(value=False, description='Fix y-axis (stress global; creep semi-fixed)')

# Global stress y-limit (worst-case)
ymax_stress_global = 1.1 * E_max * gamma0_max  # σ_max at t=0 is E*γ0

# Model functions
def stress_relaxation_maxwell(t, E, eta, gamma0):
    tau = eta / E
    return E * gamma0 * np.exp(-t / tau)

def creep_maxwell(t, E, eta, sigma0):
    return sigma0 / E + (sigma0 / eta) * t

# Plotting
def plot_maxwell(E, eta, gamma0, sigma0, T_process, T_max, time_scaled, fixed_yaxis):
    tau = eta / E
    t_end = (T_process * tau) if time_scaled else T_max
    npts = 400
    t = np.linspace(0.0, t_end, npts)

    sigma = stress_relaxation_maxwell(t, E, eta, gamma0)
    gamma = creep_maxwell(t, E, eta, sigma0)

    fig, ax = plt.subplots(1, 2, figsize=(10, 4))

    # Stress relaxation
    ax[0].plot(t, sigma, lw=2)
    ax[0].set_xlabel("Time [s]")
    ax[0].set_ylabel("Stress σ(t) [Pa]")
    ax[0].set_title("Stress relaxation (Maxwell)")
    ax[0].grid(True)
    ax[0].set_xlim(0, t_end)
    ax0_top = ax[0].secondary_xaxis('top', functions=(lambda x: x/tau if tau>0 else x, lambda x: x*tau if tau>0 else x))
    ax0_top.set_xlabel("Normalized time t/τ")

    # Creep
    ax[1].plot(t, gamma, lw=2)
    ax[1].set_xlabel("Time [s]")
    ax[1].set_ylabel("Strain γ(t) [–]")
    ax[1].set_title("Creep (Maxwell)")
    ax[1].grid(True)
    ax[1].set_xlim(0, t_end)
    ax1_top = ax[1].secondary_xaxis('top', functions=(lambda x: x/tau if tau>0 else x, lambda x: x*tau if tau>0 else x))
    ax1_top.set_xlabel("Normalized time t/τ")

    # Y-axis handling
    if fixed_yaxis:
        ax[0].set_ylim(0, ymax_stress_global)                      # fully fixed (global) stress
        gamma_max_current = (sigma0 / E) * (1.0 + (t_end / tau))   # semi-fixed creep to current window
        ax[1].set_ylim(0, 1.2 * gamma_max_current)
    else:
        for a in ax:
            a.relim(); a.autoscale_view()

    plt.tight_layout()
    plt.show()

    De = 1.0 / T_process
    print(f"τ = {tau:.3g} s   |   De = {De:.3g}")
    if time_scaled:
        print("⏱️ Time axis scaled by τ (t_end = T_process × τ).")
    else:
        print("⏱️ Time axis fixed to T_max (absolute seconds).")
    print("Top axis shows normalized time t/τ.")
    print("💡 In the creep test, γ₀ does not influence the result since the applied stress σ₀ is fixed.")

# 3-column UI layout
ui_col1 = VBox([E_slider, eta_slider])
ui_col2 = VBox([gamma0_slider, sigma0_slider])
ui_col3 = VBox([Tproc_slider, Tmax_slider, time_scaled_chk, fixed_yaxis_chk])
controls = HBox([ui_col1, ui_col2, ui_col3])

out = widgets.interactive_output(
    plot_maxwell,
    {
        'E': E_slider,
        'eta': eta_slider,
        'gamma0': gamma0_slider,
        'sigma0': sigma0_slider,
        'T_process': Tproc_slider,
        'T_max': Tmax_slider,
        'time_scaled': time_scaled_chk,
        'fixed_yaxis': fixed_yaxis_chk
    }
)

display(controls, out)


Output()

---


<details>
<summary><b>🎓 Teacher Information — click to expand</b></summary>


---

## 👩‍🏫 Teacher Information (Maxwell)

**Purpose.** Explore stress relaxation and creep with the Maxwell model (series spring–dashpot).  
**Key relation.** $ \tau = \eta / E $. Large $ \tau $ → more elastic memory; small $ \tau $ → more fluid-like.

**Use in class.**
1. Open in Colab via badge → *Run all*.
2. Vary $E$, $\eta$, $\gamma_0$, and $\sigma_0$; observe:
   - Stress relaxation: exponential decay of $\sigma(t)$.
   - Creep: linear growth of $\gamma(t)$.
3. Discuss the role of $T_\text{process}$ and Deborah number $De=\tau/T_\text{process}$.

**Notes.** Each student runs an isolated Colab session; no shared state.


</details>
